In [2]:
from langchain.document_loaders import WebBaseLoader

# Extract data from the specified URL
url = "https://brainlox.com/courses/category/technical"
loader = WebBaseLoader(url)
docs = loader.load()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
import chromadb
from langchain.vectorstores import Chroma
  # Chroma's own embeddings

# Initialize Chroma client
client = chromadb.Client()

# Define the Chroma collection
collection = client.create_collection("courses_collection")

In [4]:
docs

[Document(metadata={'source': 'https://brainlox.com/courses/category/technical', 'title': 'Brainlox: Learn technical courses.', 'description': 'Your one stop education destination!', 'language': 'zxx'}, page_content='Brainlox: Learn technical courses.Courses TechnicalAcademicLanguageMusicLifestyleBook a Free Demo NowSign InFAQContact UsHomeCoursesCoursesWe found great courses available for you$30per sessionLEARN SCRATCH PROGRAMING\nScratch Course is the foundation of coding and is a building block of a coding journey. If you want 16 LessonsView Details$30per sessionLEARN CLOUD COMPUTING BASICS-AWS\nIn this course we are going to cover the basics and the most important services on AWS,\nAt the end  20 LessonsView Details$30per sessionLEARN MOBILE DEVELOPMENT\nMobile application development is the process of creating software applications that run on a mobil 24 LessonsView Details$30per sessionLEARN CORE JAVA PROGRAMMING ONLINE\nJava is a very popular high-level, class-based, object-orie

In [5]:


# Iterate through the loaded docs and extract relevant course information
for doc in docs:
    content = doc.page_content
    # Split the page content by each course using patterns such as "$30per session"
    # or based on the courses' structure.
    course_data = content.split('$30per session')
    
    for course in course_data:
        if len(course.strip()) > 0:  # Ensure the course data is not empty
            courses.append(course.strip())

# Now, courses is a list of individual course descriptions

NameError: name 'courses' is not defined

In [6]:
course_data

['Brainlox: Learn technical courses.Courses TechnicalAcademicLanguageMusicLifestyleBook a Free Demo NowSign InFAQContact UsHomeCoursesCoursesWe found great courses available for you',
 'LEARN SCRATCH PROGRAMING\nScratch Course is the foundation of coding and is a building block of a coding journey. If you want 16 LessonsView Details',
 'LEARN CLOUD COMPUTING BASICS-AWS\nIn this course we are going to cover the basics and the most important services on AWS,\nAt the end  20 LessonsView Details',
 'LEARN MOBILE DEVELOPMENT\nMobile application development is the process of creating software applications that run on a mobil 24 LessonsView Details',
 'LEARN CORE JAVA PROGRAMMING ONLINE\nJava is a very popular high-level, class-based, object-oriented programming language that is design 41 LessonsView Details',
 'LEARN ROBOTICS\nYou can open all kinds of doors for advancement in so many careers with a basic understanding of el 25 LessonsView Details',
 'LEARN JAVASCRIPT\nJavaScript is the most

In [7]:
len(course_data)

63

In [8]:
from chromadb.utils import embedding_functions
default_ef = embedding_functions.DefaultEmbeddingFunction()
embeddings_list = default_ef(course_data) 
course_ids = [f"course_{i}" for i in range(len(course_data))]

In [10]:
embeddings_2d = [embedding.tolist() for embedding in embeddings_list]

In [11]:
len(embeddings_2d)

63

In [12]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from sentence_transformers import SentenceTransformer


/media/vinayak/New Volume/Prgramming/ml/nsaml/as/scrsap/as2/chat_bot/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
embedding_model = SentenceTransformer("BAAI/bge-small-en")
class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        return embedding_model.encode(input).tolist()

In [14]:
# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [19]:
collection = chroma_client.get_or_create_collection(name="courses_collection", embedding_function=MyEmbeddingFunction())

In [20]:
collection.add(
    documents=course_data,
    ids=[str(i) for i in range(len(course_data))]
)

In [21]:
collection.get()

{'ids': ['0',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '49',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '60',
  '61',
  '62'],
 'embeddings': None,
 'documents': ['Brainlox: Learn technical courses.Courses TechnicalAcademicLanguageMusicLifestyleBook a Free Demo NowSign InFAQContact UsHomeCoursesCoursesWe found great courses available for you',
  'LEARN SCRATCH PROGRAMING\nScratch Course is the foundation of coding and is a building block of a coding journey. If you want 16 LessonsView Details',
  'LEARN CLOUD COMPUTING BASICS-AWS\nIn this course we are going to cover the basics and the most important services on AWS,\

In [22]:
collection.get(ids=['1'])

{'ids': ['1'],
 'embeddings': None,
 'documents': ['LEARN SCRATCH PROGRAMING\nScratch Course is the foundation of coding and is a building block of a coding journey. If you want 16 LessonsView Details'],
 'uris': None,
 'data': None,
 'metadatas': [None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [24]:
query_text = "Learn python"
results = collection.query(
    query_texts=[query_text],
    n_results=3  # Retrieve top 3 matches
)

print("Search Results:", results)

Search Results: {'ids': [['35', '33', '8']], 'embeddings': None, 'documents': [['Python Playground: Create a Tic Tac Toe GameAn interactive and hands-on tutorial designed to help learners build their programming skills while  8 LessonsView Details', 'Python Playground: Create Your Own Snake GameThrough this course, participants will learn the fundamentals of Python programming language, as wel 8 LessonsView Details', 'PYTHON PROGRAMMING-BEGINNER\nPython is a language with simple syntax, and a powerful set of libraries. It has a rich programming 16 LessonsView Details']], 'uris': None, 'data': None, 'metadatas': [[None, None, None]], 'distances': [[0.1973507013621699, 0.19918437032878522, 0.2065759517655007]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
